In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
plt.style.use('fivethirtyeight')

In [ ]:
# Import relevant dataframes:

main_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Main_Data\Main_Table_10182019.csv")
print(len(main_table))
faculty_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_08272019.csv")
print(len(faculty_table))
# skill_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Skills_Data\Skill_Table_06072019.csv")
# print(len(skill_table))

In [ ]:
def title_swap(string):
    dictionary = {'Community and Social Services':'Counseling and Religious Life',
                  'Customer and Client Support':'Online Support and University Information',
                  'Hospitality, Food, and Tourism':'Event Management and Hospitality', 
                  'Planning and Analysis':'Analysis', 
                  'Curriculum and Instructional Designer / Developer':'Curriculum and Instructional Designer', 
                  'Special Education Teacher':'Accessibility and Disability Services', 
                  'Teaching Assistant':'Faculty Support', 
                  'Tutor':'Academic Tutor',
                  'Clerical and Administrative':'Administrative',
                  'na' : 'Uncategorized'}
    if(string in dictionary):
        return dictionary[string]
    else:
        return string
    
def val_year(year1, year2):
    if(year1 >= year2):
        if(year1>year2):
            temp = year1
            year1 = year2
            year2 = temp
        else:
            raise ValueError('Years cannot be the same.')
            
    return (year1, year2)

def cat_validate(actual, desired_list):
    return actual in desired_list

In [ ]:
post_doc = main_table[main_table['Job Title'].str.contains('postdoc', case=False, na=False) |
                      main_table['Job Title'].str.contains('post doc', case=False, na=False)|
                      main_table['Job Title'].str.contains('post-doc', case=False, na=False)]
print(len(post_doc))

In [ ]:
mask = faculty_table[faculty_table['Post-Doctoral'] == 1][['Job ID']]
print(len(mask))

In [ ]:
boolean_vec = (main_table['Job Title'].str.contains('postdoc', case=False, na=False) |
                      main_table['Job Title'].str.contains('post doc', case=False, na=False)|
                      main_table['Job Title'].str.contains('post-doc', case=False, na=False))

main_table['post_doc'] = boolean_vec + 0

faculty_table = faculty_table.merge(main_table[['Job ID', 'post_doc']], on='Job ID', how='inner')

In [ ]:
def boolean(x):
    if x > 0:
        return 1
    else:
        return 0

faculty_table['Post-Doctoral'] = faculty_table['post_doc'].apply(boolean)

In [ ]:
faculty_table['Post-Doctoral'].sum()

In [ ]:
def breakout(df, year, category, sig_level=0.01):
    df = df[df['Year']==year]
    num = df['Job ID'].nunique()
    arr = np.zeros(num)
    p = 1
    i = 1
    while(i <= num):
        arr[:i] = 1
        p = stats.ttest_1samp(arr, popmean=0).pvalue
        if p/2 <= sig_level:
            print(num, '->', i)
            break
        i+=1
    
    cat = pd.DataFrame(df[category].value_counts()).reset_index()
    cat = cat.rename(columns={category:'count'})
    cat = cat[cat['count']>=i]
    cat['inc'] = np.true_divide(cat['count'], num)
    
    return cat

def growth(df1, df2):
    df = df1.merge(df2, on='index', how='outer', indicator=True)
    df = df[df['_merge']=='both']
    df['growth'] = np.true_divide(df['inc_y'] - df['inc_x'], df['inc_x'])
    return df.sort_values('growth', ascending=False).reset_index(drop=True)
    
mask = faculty_table[faculty_table['Post-Doctoral'] == 1][['Job ID']]
post = main_table.merge(mask, on='Job ID', how='inner')

inst07 = breakout(post, 2007, 'IPEDS Institution Name')
inst17 = breakout(post, 2017, 'IPEDS Institution Name')

inst_g = growth(inst07, inst17)

zone07 = breakout(post, 2007, 'BEA_Zone')
zone17 = breakout(post, 2017, 'BEA_Zone')

zone_g = growth(zone07, zone17)

In [ ]:
inst_g

In [ ]:
zone_g

In [ ]:
doc07 = breakout(faculty_table, 2007, 'Post-Doctoral')
doc17 = breakout(faculty_table, 2017, 'Post-Doctoral')

growth(doc07, doc17)

In [ ]:
def t_test(n):
    arr = np.zeros(n)
    arr[:4] = 1

    mean = np.mean(arr)
    diff = (arr - mean)**2
    std = math.sqrt(diff.sum()/(n-1))
    se = std/math.sqrt(n)
    t = (mean - 0)/se
    print(mean, std, se, t)

# for n in range(100):
#     if(n<4):
#         continue
n=4000000
arr = np.zeros(n)
arr[:4] = 1
test = stats.ttest_1samp(arr, popmean=0)
p = test.pvalue
t = test.statistic
print(n)
print('\t', t, p)


In [ ]:
stats.ttest_1samp(arr, popmean=0)